##### Steps #####

1. Detect and Load dataset
2. Keras
3. Scikit
4. Kfold
5. ROC curve
6. Classification report
7. Write back the results

---

##### Detecting and loading data #####

In [50]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('../data/client_secret.json', scope)

gc = gspread.authorize(credentials)

sh = gc.open('Dataset details')
worksheet = sh.get_worksheet(0)
dataset_list = worksheet.col_values(2)
activation_list = worksheet.col_values(3)
dataset_list

# for data in dataset_list:
#     if data follows a specific format then:
#         process the dataset
#         Create a list of dataframes
activation_function = activation_list[2]
data_url = dataset_list[2]
activation_function

'Logistic regression'

In [49]:
from sklearn.model_selection import train_test_split

data = pd.read_csv(data_url, delimiter=",", header=None, index_col=False)
class_name,index = np.unique(data.iloc[:,-1],return_inverse=True)
data.iloc[:,-1] = index
data = data.loc[data[4] != 2]
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

---

##### Keras model #####

In [52]:
### Logistic regression using DNN ###
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

np.random.seed(7)

# Defining model #
if activation_function == "Logistic regression":
    activation = "sigmoid"
input_dim = X.shape[1]
epoch = 200 # Add in the sheet
batch_size = 10 # Add in the sheet


model = Sequential()
model.add(Dense(1,input_dim=input_dim,activation=activation))

# Compile the model #

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model #

# model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=10)
model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)
# Evaluate the model #

scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.


60/60 [==============================] - 0s 418us/step

acc: 93.33%


Model details to be added
1. Score/Accuracy
2. bias_initializer
3. kernel_regularizer
4. bias_regularizer
5. 

In [54]:
config = model.get_config()
config

{'name': 'sequential_1',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'batch_input_shape': (None, 4),
    'dtype': 'float32',
    'units': 1,
    'activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}]}